In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re
from shapely.geometry import MultiPolygon, Polygon, Point
from sklearn.cluster import KMeans
from geopy.distance import distance
from ast import literal_eval
import math 
from geopy.distance import distance
import pyproj
import ast
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
from matplotlib.backends.backend_pdf import PdfPages


In [2]:
def plot_data(df, title):
    
    # Separate x and y values from the coordinate tuples and get masses of each point
    x_values = [row['Biomass location'][0] for _, row in df.iterrows()]
    y_values = [row['Biomass location'][1] for _, row in df.iterrows()]
    mass_values = df['Mass'].values

    # To plot the coordinates in WGS84-system
    
    # Define the WGS84 and EUREF-FIN coordinate systems
    wgs84 = pyproj.CRS('EPSG:4326')
    euref_fin = pyproj.CRS('EPSG:3067')
    
    # Transform the location to WGS84 coordinates
    transformer = pyproj.Transformer.from_crs(euref_fin, wgs84, always_xy=True)
    x_values, y_values = transformer.transform(x_values,y_values)
    
    # Plot the data with the colors based on mass
    fig, ax = plt.subplots(figsize=(8, 8))
    sc = ax.scatter(x_values, y_values, c=mass_values, cmap='viridis', s=100)
    plt.xlabel('Longitude', fontsize=14)
    plt.ylabel('Latitude', fontsize=14)
    plt.title(' '.join(title.split(' ')[:4]) + '\n' + ' '.join(title.split(' ')[4:]), fontsize=16)
    plt.tick_params(labelsize=12)
    
    # Add a colorbar to show the mass values
    cbar = plt.colorbar(sc)
    cbar.ax.set_ylabel('Mass (tonns)', fontsize=14)
    
    path = './pdfs/Biomasses/'
    
    if 'within the given radius' in title:
        path = './pdfs/Biomasses within the given radius to the biogas facility/'
    
    fig.savefig(path + title+'.pdf')
    
    plt.show()


In [3]:
def plot_clusters(df,weighted_avg_centroids,sum_cluster_masses,pdfname):
    
    masses_in_truck_capacity = np.array(sum_cluster_masses) / 45
    
    # Set up a colormap to color the centroids based on their sum of masses
    cmap = plt.get_cmap('Reds')
    norm = mcolors.LogNorm(vmin=np.array(masses_in_truck_capacity).min(), vmax=np.array(masses_in_truck_capacity).max())
    #norm = plt.Normalize(np.array(masses_in_truck_capacity).min(), np.array(masses_in_truck_capacity).max())
    fig, ax = plt.subplots(figsize=(10,10))

    locations = df['Biomass location'].tolist()
    locations = np.array(locations)
    centroids = np.array(weighted_avg_centroids)
    
    # To plot the coordinates in WGS84-system,
    # define the WGS84 and EUREF-FIN coordinate systems
    wgs84 = pyproj.CRS('EPSG:4326')
    euref_fin = pyproj.CRS('EPSG:3067')
    
    # Transform the location to WGS84 coordinates
    transformer = pyproj.Transformer.from_crs(euref_fin, wgs84, always_xy=True)
    x_values, y_values = transformer.transform(locations[:, 0],locations[:, 1])
    x_centroids, y_centroids = transformer.transform(centroids[:, 0],centroids[:, 1])

    # Plotting the data with the centroids

    # scatter plot of data points and their clusters
    plt.scatter(x_values, y_values, c=df['Cluster'])

    # scatter plot of cluster centroids colored by their sum of masses
    plt.scatter(x_centroids, y_centroids, marker='.', s=200, linewidths=3, c=masses_in_truck_capacity, cmap=cmap, norm=norm)

    # Add a colorbar to show the logarithmic scale colormap legend
    cbar = plt.colorbar()
    cbar.ax.set_ylabel('Masses in truckloads (45 tons)')
    cbar.set_ticks([1, 10, 100, 1000])  # Customize the ticks on the colorbar
    
    plt.xlabel('Longitude', fontsize=14)
    plt.ylabel('Latitude', fontsize=14)
    plt.title('Pickup sites for the file '+' '.join(filename.split(' ')[:4]) + '\n' + ' '.join(filename.split(' ')[4:]), fontsize=16)

    path = './pdfs/Raw pickup sites/'

    if 'Thinned' in pdfname:
        path = './pdfs/Thinned out pickup sites/'
    
    fig.savefig(path + pdfname +filename+'.pdf')
    plt.show()

In [4]:
def load_and_process_data(filename):
    # Read the CSV file and specify the data types of the columns
    dtypes = {'Area': str}
    df = pd.read_csv(filename, header=0, sep=';', dtype=dtypes)

    # Replace characters in the Area column and split it into separate columns
    replacements = {'(': '', ')': '', 'MULTIPOLYGON': ''}
    for index1, row in df.iterrows():
        input_string = str(df['Area'][index1])
        for old, new in replacements.items():
            input_string = input_string.replace(old, new)
        coordinates = input_string.split(',')
        for index2, coordinate in enumerate(coordinates):
            df.loc[index1,'Coordinate '+ str(index2)]=coordinate

    # Calculate the mean points of biomass potentials and replace the coordinates with them
    df2 = df[['Coordinate 0','Coordinate 1','Coordinate 2','Coordinate 3']]
    df2 = df2.apply(lambda row: tuple(map(lambda x: np.sum(x)/len(x), zip(*[map(float, str(c).split()) for c in row]))), axis=1)
    df['Biomass location'] = df2

    # Clean the dataframe
    df = df.loc[:, ['Mass','Biomass location']]
    mask = df['Biomass location'].apply(is_2d_tuple)
    df = df[mask]
    
    return df


In [5]:
def clean_data(df,radius,facilityloc):

    # Clean the data (df) from the rows that are further away than the assumed maximum distance (radius)
    # from the biogas facility locations (facilityloc). 

    # Check input parameters
    if not isinstance(facilityloc, tuple) or len(facilityloc) != 2 or \
       not all(isinstance(x, (int, float)) for x in facilityloc):
        raise ValueError('facilityloc must be a tuple with two numeric elements')

    # Define the WGS84 and EUREF-FIN coordinate systems
    wgs84 = pyproj.CRS('EPSG:4326')
    euref_fin = pyproj.CRS('EPSG:3067')
    
    # Transform the facility location to EUREF-FIN coordinates
    transformer = pyproj.Transformer.from_crs(wgs84, euref_fin, always_xy=True)
    facility_euref_x, facility_euref_y = transformer.transform(facilityloc[1], facilityloc[0])
    facility_loc = (facility_euref_x, facility_euref_y)
 
    
    # Calculate the distances
    distances = df['Biomass location'].apply(lambda loc: math.sqrt(pow(facility_loc[0]-loc[0],2)+pow(facility_loc[1]-loc[1],2))/1000)

    # Filter the data
    df = df.loc[distances <= radius].reset_index(drop=True)
    
    # Remove zero masses from data
    df = df.loc[(df != 0).all(axis=1)]

    return df

In [6]:
def df_cluster_masses_and_locations(df):

    # Getting masses and locations for each cluster

    cluster_masses = [[] for _ in range(len(df['Cluster'].unique()))]
    cluster_locations = [[] for _ in range(len(df['Cluster'].unique()))]

    for index, row in df.iterrows():
        cluster_masses[row['Cluster']].append(row['Mass'])
        cluster_locations[row['Cluster']].append(row['Biomass location'])

    return cluster_masses, cluster_locations

In [7]:
def cluster_sums_and_weight_avg_centroids(df,cluster_masses,cluster_locations):
    # Calculate the weighted average centroid for dataframe's clusters:

    weighted_avg_centroids = []
    sum_cluster_masses = []

    for i in range(len(df['Cluster'].unique())):
        sum_x = np.sum([mass * loc[0] for mass, loc in zip(cluster_masses[i], cluster_locations[i])])
        sum_y = np.sum([mass * loc[1] for mass, loc in zip(cluster_masses[i], cluster_locations[i])])
        weighted_avg_centroids.append((sum_x / np.sum(cluster_masses[i]), sum_y / np.sum(cluster_masses[i])))
        sum_cluster_masses.append(np.sum(cluster_masses[i]))

    return weighted_avg_centroids, sum_cluster_masses

In [8]:
def is_2d_tuple(val):
    
    # Check whether val given as parameter is in expected form of tuple. is used by load_and_process data.
    
    return isinstance(val, tuple) and len(val) == 2 and all(isinstance(x, (int, float)) for x in val)

In [9]:
facility_loc = None 
greatest_distance = None

def set_facility_loc_and_rad():
    global facility_loc
    global greatest_distance  
    user_input = input("Give the coordinates of facility in inspection as a tuple: ")
    trimmed_input = user_input.strip("())")
    coordinates = trimmed_input.split(",")
    coordinates = [float(x) for x in coordinates]
    facility_loc = tuple(coordinates)
    greatest_distance = int(input("Give greatest distance of what biomasses are included: "))

In [10]:
def thin_clusters(df,tire_thickness,tires,cluster_divisor_coefficient,facility_loc):
    
    # Define the WGS84 and EUREF-FIN coordinate systems
    wgs84 = pyproj.CRS('EPSG:4326')
    euref_fin = pyproj.CRS('EPSG:3067')
    
    # Transform the facility location to EUREF-FIN coordinates
    transformer = pyproj.Transformer.from_crs(wgs84, euref_fin, always_xy=True)
    facility_euref_x, facility_euref_y = transformer.transform(facility_loc[1], facility_loc[0])
    facility_loc = (facility_euref_x, facility_euref_y)
    
    # Calculate the distances
    distances = df['Biomass location'].apply(lambda loc: math.sqrt(pow(facility_loc[0]-loc[0],2)+pow(facility_loc[1]-loc[1],2))/1000)
    #i_values = {}  # Create an empty dictionary to store the separated dataframe names and their values
    i_values = [None]*tires
    
    for i in range(tires,0,-1): # Reversed looping for handling the tires in a smarter way
        
        # Reordering the data to subsets of tires
        i_values[tires-i] = df[(tire_thickness*(i-1) <= distances) & (distances <= tire_thickness*i)]  # Assign the filtered data to the corresponding tire

        # Coordinates with mass coefficients of a data subset 
        coords = i_values[tires-i]['LocationWithMassCoefficient'].explode().tolist()

        LocationMassCoords = [(coords[i], coords[i+1]) for i in range(0, len(coords)-1, 2)]
        LocationMassCoords = np.array(LocationMassCoords)

        # CLustering the data of a subset with k-value defined by cluster_divisor_coefficient. Thinning the cluster-areas from 
        # greater distance. Saving new clusters to ThinnedCLusters columns
        previous_kluster_amount = len(i_values[tires-i]['Cluster'].unique()) # NOTE: IN ORIGINAL DATA, ONE CLUSTER MAY BE LOCATED TO MULTIPLE TIRE
                                                                             # THIS SHOULD BE TAKEN IN NOTICE IN DECIDING cluster_divisor_coefficient     
        kmeans = KMeans(n_clusters=int(previous_kluster_amount*pow(cluster_divisor_coefficient,i-1)),n_init='auto')
        kmeans.fit(LocationMassCoords)

        i_values[tires-i]['Cluster'] = kmeans.predict(i_values[tires-i]['Biomass location'].tolist())

    return i_values    

In [11]:
def unite_dataframes_of_thinned_cluster_areas(dfs):

    # Assuming your dataframes are stored in a list called "dfs"
    
    for i, df in enumerate(dfs):
        if i == 0:
            combined_df = df
            continue
        
        clustersAdded = int(len(combined_df['Cluster'].unique()))
        
        df['Cluster'] = df['Cluster']+clustersAdded
        
        combined_df = pd.concat([combined_df,df])
        
    return combined_df

In [12]:
def cluster_biomasses_with_KMeans(df,k):
    
    # To duplicate the coordinate points based on their masses to consider
    # the geographical distribution of masses in clustering.
    
    df['MassCoefficient'] = df['Mass'].apply(round).astype(int).clip(lower=1)
    df['LocationWithMassCoefficient'] = df['Biomass location'] * df['MassCoefficient']
    coords = df['LocationWithMassCoefficient'].explode().tolist()
    LocationMassCoords = [(coords[i], coords[i+1]) for i in range(0, len(coords)-1, 2)]
    LocationMassCoords = np.array(LocationMassCoords)    

    # fitting the mass-distribution fixed data with KMeans
    
    kmeans = KMeans(n_clusters=k,n_init='auto')
    kmeans.fit(LocationMassCoords)
    
    locations = df['Biomass location'].tolist()
    locations = np.array(locations)

    # Making predictions of cluster for each coordinate point 
    df['Cluster'] = kmeans.predict(locations)
    
    return df, kmeans

In [14]:
filename = input("Enter biomassdata filename: ") + '.csv'
folder = input("Enter the name of the folder where the file is located: ")
path = './datasets/'
set_facility_loc_and_rad()
df = load_and_process_data(path+'/'+folder+'/'+filename)
file = filename.split('.',1)[0]
# Coordinates for Envor's facility: (60.929833, 23.369694)

Enter biomassdata filename: Nautojen lietelanta varastosta 2015
Enter the name of the folder where the file is located: Lietelannat
Give the coordinates of facility in inspection as a tuple: (60.929833, 23.369694)
Give greatest distance of what biomasses are included: 40


C:\Users\K-A-S\AppData\Local\Temp\ipykernel_15016\700681997.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, header=0, sep=';', dtype=dtypes)
C:\Users\K-A-S\AppData\Local\Temp\ipykernel_15016\700681997.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index1,'Coordinate '+ str(index2)]=coordinate
C:\Users\K-A-S\AppData\Local\Temp\ipykernel_15016\700681997.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index1,'Coordinate '

C:\Users\K-A-S\AppData\Local\Temp\ipykernel_15016\700681997.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index1,'Coordinate '+ str(index2)]=coordinate
C:\Users\K-A-S\AppData\Local\Temp\ipykernel_15016\700681997.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index1,'Coordinate '+ str(index2)]=coordinate
C:\Users\K-A-S\AppData\Local\Temp\ipykernel_15016\700681997.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\K-A-S\anaconda3\envs\JODA\lib\site-packages\pandas\core\internals\blocks.py", line 973, in setitem
    casted = np_can_hold_element(values.dtype, value)
  File "C:\Users\K-A-S\anaconda3\envs\JODA\lib\site-packages\pandas\core\dtypes\cast.py", line 2041, in np_can_hold_element
    raise LossySetitemError
pandas.core.dtypes.cast.LossySetitemError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\K-A-S\anaconda3\envs\JODA\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\K-A-S\AppData\Local\Temp\ipykernel_15016\470240604.py", line 5, in <module>
    df = load_and_process_data(path+'/'+folder+'/'+filename)
  File "C:\Users\K-A-S\AppData\Local\Temp\ipykernel_15016\700681997.py", line 14, in load_and_process_data
    df.loc[index1,'Coordinate '+ str(index2)]=coordinate
  Fil

In [15]:
plot_data(df,'Biomasses of ' + file)

NameError: name 'df' is not defined

In [ ]:
df = clean_data(df,greatest_distance,facility_loc)

df

In [ ]:
plot_data(df,file+' within the given radius to the biogas facility')

In [ ]:
# Tsekkaa paperi GIS-data related route optimization, hierarchical clustering, location
# optimization, and kernel density methods are useful for promoting
# distributed bioenergy plant planning in rural areas
# K. Laasasenahoa*, A. Lensub*, R. Lauhanenc
# and J. Rintalaa

# Clustering the data with K-means. Utilzing the elbow method to decide optimal value for k.
# The basic idea in elbow method is to plot the within-cluster 
# sum of squares (WCSS) against the number of clusters, and choose the number of clusters 
# at the "elbow" point where the rate of decrease in WCSS slows down significantly.

# To be used in consideration of mass distribution. Each coordinate will be multiplied 
# with their MassCoefficient in clustering
# to give more weight to the points with greater mass. 

df['MassCoefficient'] = df['Mass'].apply(round).astype(int).clip(lower=1)
df['LocationWithMassCoefficient'] = df['Biomass location'] * df['MassCoefficient']

coords = df['LocationWithMassCoefficient'].explode().tolist()
LocationMassCoords = [(coords[i], coords[i+1]) for i in range(0, len(coords)-1, 2)]
LocationMassCoords = np.array(LocationMassCoords)

'''
k_vals = list(range(1,100))
wcss = []
labels=[]

for i in k_vals:    
    kmeans = KMeans(n_clusters=i,n_init='auto')
    kmeans.fit(LocationMassCoords)
    wcss.append(kmeans.inertia_)
    labels.append(kmeans.predict(LocationMassCoords))
''' 

In [ ]:
# Plot the WCSS values against the number of clusters
'''
plt.plot(range(1, 100), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()
'''

In [ ]:
# Based on the graph above, close to optimal and good enough k-values for this datasets is 60.
# Let's plot the data with 60 clusters.
# Plotting the data

# Because of the later trimming data from the clusters far away from the facility, initialize clustering with K=200

#KCLUSTERS = 200

#kmeans = KMeans(n_clusters=KCLUSTERS,n_init='auto')
#kmeans.fit(LocationMassCoords)
#labels = kmeans.predict(LocationMassCoords)
#plt.scatter(LocationMassCoords[:,0],LocationMassCoords[:,1],c=labels)

In [ ]:
df, kmeans = cluster_biomasses_with_KMeans(df,200)

In [ ]:
df

In [ ]:
cluster_masses, cluster_locations =  df_cluster_masses_and_locations(df)

In [ ]:
weighted_avg_centroids, sum_cluster_masses = cluster_sums_and_weight_avg_centroids(df,cluster_masses, cluster_locations)

In [ ]:
plot_clusters(df,weighted_avg_centroids,sum_cluster_masses,'Raw pickup sites for the file ')

In [ ]:
thinned_and_clustered_tires = thin_clusters(df,10,5,0.7,facility_loc)

In [ ]:
thinned_area = unite_dataframes_of_thinned_cluster_areas(thinned_and_clustered_tires)

In [ ]:
thinned_area

In [ ]:
thinned_cluster_masses, thinned_cluster_locations =  df_cluster_masses_and_locations(thinned_area)

In [ ]:
thinned_weighted_avg_centroids, thinned_sum_cluster_masses = cluster_sums_and_weight_avg_centroids(thinned_area,thinned_cluster_masses, thinned_cluster_locations)

In [ ]:
plot_clusters(thinned_area,thinned_weighted_avg_centroids, thinned_sum_cluster_masses,'Thinned out pickup sites for the file ')

In [ ]:
# SOME COORDINATE-CONVERSION HANDLING

# define the ETRS-TM35FIN projection

# CHECK LATER IF THIS IS NEEDED AT ALL

etrs_tm35fin = pyproj.Proj("+proj=utm +zone=35 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs")

# define a function to convert EUREF-FIN coordinates to lon-lat
def euref_fin_to_lon_lat(row):
    lon, lat = etrs_tm35fin(row['ClusterLocations'][0], row['ClusterLocations'][1], inverse=True)
    return pd.Series({'lon': lon, 'lat': lat})

In [ ]:
# CLUSTERS TO GEOJSONS

df = pd.DataFrame({'ClusterLocations': weighted_avg_centroids, 'Clustermasses': sum_cluster_masses}, index=range(1, len(weighted_avg_centroids)+1))

df[['lon','lat']] = df.apply(euref_fin_to_lon_lat,axis=1)

geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
gdf = gdf.drop('ClusterLocations',axis=1) # Not needed anymore, these are in the wrong form of coordinates

## save the GeoDataFrame to a GeoJSON file
path = './geojsons/'
gdf.to_file(path + file+'.geojson', driver='GeoJSON')